In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch.nn import Linear
import torch.nn as nn
import torch.nn.functional as F


Bad key "text.kerning_factor" on line 4 in
C:\Users\Joshua\Anaconda3\envs\pytorch\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df_raw = pd.read_csv('./data/total_df.csv')

In [3]:
df = df_raw[(df_raw['REG_YYMM'] == 202001) | (df_raw['REG_YYMM'] == 202002) | (df_raw['REG_YYMM'] == 202003)]
df.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,REG_YYMM,CSTMR_CNT,AMT,CNT,CV,AVG_TEMP,AVG_MAX_TEMP,AVG_MIN_TEMP,MONTH_PRECIPITATION
9533,강원,강릉시,건강보조식품 소매업,20s,2,1,202001,3,345000,3,0,3.4,7.2,0.3,148.1
9534,강원,강릉시,건강보조식품 소매업,30s,1,2,202001,3,1903450,3,0,3.4,7.2,0.3,148.1
9535,강원,강릉시,건강보조식품 소매업,30s,2,2,202001,14,1520500,15,0,3.4,7.2,0.3,148.1
9536,강원,강릉시,건강보조식품 소매업,40s,1,3,202001,9,1239200,9,0,3.4,7.2,0.3,148.1
9537,강원,강릉시,건강보조식품 소매업,40s,1,4,202001,3,606700,4,0,3.4,7.2,0.3,148.1


In [4]:
df.REG_YYMM.unique()

array([202001, 202002, 202003], dtype=int64)

# Label Encoding

In [5]:
le = LabelEncoder()
# le = le.fit(df['CARD_SIDO_NM'])
# df['CARD_SIDO_NM'] = le.transform(df['CARD_SIDO_NM'])

In [6]:
df.drop(['CV', 'AVG_TEMP',
       'AVG_MAX_TEMP', 'AVG_MIN_TEMP', 'MONTH_PRECIPITATION'], axis=1, inplace=True)

In [7]:
df.columns

Index(['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'REG_YYMM', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [8]:
categorical_cols = ['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE']

In [9]:
for col in categorical_cols:
    le = le.fit(df['{}'.format(col)])
    df['{}'.format(col)] = le.transform( df['{}'.format(col)])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239569 entries, 9533 to 1225753
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   CARD_SIDO_NM  239569 non-null  int32
 1   CARD_CCG_NM   239569 non-null  int32
 2   STD_CLSS_NM   239569 non-null  int32
 3   AGE           239569 non-null  int32
 4   SEX_CTGO_CD   239569 non-null  int64
 5   FLC           239569 non-null  int64
 6   REG_YYMM      239569 non-null  int64
 7   CSTMR_CNT     239569 non-null  int64
 8   AMT           239569 non-null  int64
 9   CNT           239569 non-null  int64
dtypes: int32(4), int64(6)
memory usage: 16.4 MB


# Preprocessing

In [11]:
# 변수명 지정
X_cols = list(df.columns)
X_cols.remove('AMT')

In [12]:
X = df[X_cols]
y = df['AMT']

In [13]:
# X 변수 Scale 적용
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [15]:
X_train.shape

(167698, 9)

In [20]:
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [21]:
y_test

tensor([3.9095e+05, 2.2025e+07, 1.3000e+05,  ..., 1.1725e+06, 7.5135e+08,
        1.5158e+06])

In [22]:
X_train

tensor([[ 0.4270, -0.5637,  0.8158,  ...,  1.2468, -0.1075, -0.1002],
        [ 2.0589,  1.4328,  0.1859,  ..., -1.1988, -0.1852, -0.1257],
        [-1.2049,  1.7655,  0.8158,  ..., -1.1988, -0.1758, -0.1250],
        ...,
        [ 0.9709, -0.7940,  0.2759,  ...,  0.0240, -0.2105, -0.1367],
        [-0.9329,  0.4602,  0.2759,  ..., -1.1988, -0.2116, -0.1372],
        [-1.4769,  0.6905,  1.5357,  ...,  1.2468, -0.2116, -0.1372]])

In [19]:
# X_train = torch.Tensor(X_train)
# X_test = torch.Tensor(X_test)
# y_train = torch.Tensor(y_train.values)
# y_test = torch.Tensor(y_test.values)

TypeError: new(): data must be a sequence (got builtin_function_or_method)

# NN Model

In [24]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(9, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num = 1
        for i in size:
            num *= i
        return num

In [25]:
model = NN()
print(model)

NN(
  (fc1): Linear(in_features=9, out_features=1, bias=True)
)


In [26]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

In [27]:
nb_epochs = 1
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(X_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 112490476816 bytes. Buy new RAM!


In [ ]:
def train_model(model, train_data, train_labels, test_data=None, test_labels=None, num_epochs=250, lr=1e-3):
    loss_fn = nn.MSELoss()
    
    optimiser = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4)
    num_epochs = num_epochs
    
    train_hist = np.zeros(num_epochs)
    test_hist = np.zeros(num_epochs)
    
    for t in range(num_epochs):
        y_pred = model(X_train)
        loss = loss_fn(y_pred.float(), y_train)
        
        if test_data is not None:
            with torch.no_grad():
                y_test_pred = model(X_test)
                test_loss = loss_fn(y_test_pred.float(), y_test)
            test_hist[t] = test_loss.item()
            
            if t % 10 == 0:
                print(f'Epoch {t} train loss: {round(loss.item(),4)} test loss: {round(test_loss.item(),4)}')
        elif t % 10 == 0:
            print(f'Epoch {t} train loss: {loss.item()}')
            
        train_hist[t] = loss.item()
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
    return model.eval(), train_hist, test_hist

In [99]:
# Hyper-parameter 설정
n_features=X_train.shape[-1]
n_hidden=4
n_layers=2
lr=1e-4
num_epochs=1

In [28]:
# Training Model
model = NN()
model, train_hist, test_hist = train_model(model, X_train, y_train, X_test, y_test, num_epochs=num_epochs, lr=lr)

AttributeError: 'Linear' object has no attribute 'size'